# CUDA C

GPT : https://chatgpt.com/share/6728f8c2-7aac-8002-b253-397bfeb02832

For writing CUDA program write this in the notebook:

In [ ]:
!nvidia-smi

Mon Nov  4 19:42:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git

Cloning into 'cuda-samples'...
remote: Enumerating objects: 19507, done.
remote: Counting objects: 100% (4922/4922), done.
remote: Compressing objects: 100% (658/658), done.
remote: Total 19507 (delta 4565), reused 4336 (delta 4264), pack-reused 14585 (from 1)
Receiving objects: 100% (19507/19507), 133.38 MiB | 11.91 MiB/s, done.
Resolving deltas: 100% (17225/17225), done.
Updating files: 100% (4026/4026), done.


In [ ]:
!cd cuda-samples/Samples/1_Utilities/deviceQuery &&make

/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda/bin/nvcc -ccbin g++ -m64 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery deviceQuery.o 
mkdir -p

In [ ]:
!cd cuda-samples/Samples/1_Utilities/deviceQuery &&ls
!cuda-samples/Samples/1_Utilities/deviceQuery/./deviceQuery

deviceQuery	 deviceQuery_vs2017.sln      deviceQuery_vs2019.vcxproj  Makefile
deviceQuery.cpp  deviceQuery_vs2017.vcxproj  deviceQuery_vs2022.sln	 NsightEclipse.xml
deviceQuery.o	 deviceQuery_vs2019.sln      deviceQuery_vs2022.vcxproj  README.md
cuda-samples/Samples/1_Utilities/deviceQuery/./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmph373i958".


1. Hello World Program (helloWorld.cu)

In [ ]:
%%writefile helloWorld.cu
#include <stdio.h>
#include <cuda.h>

const int a = 5;

__global__ void my_kernel() {
  printf("Hello, World! <-- GPU\n");
  printf("Value printed from GPU: %d\n", a);
}

int main() {
  printf("Hello, World! <-- CPU\n" );
  my_kernel<<<1, 1>>>();
  printf("Value printed from GPU: %d\n", a);
  cudaDeviceSynchronize();
  return 0;
}

Writing helloWorld.cu


In [ ]:
!nvcc helloWorld.cu -o helloWorld
!./helloWorld

Hello, World! <-- CPU
Value printed from GPU: 5
Hello, World! <-- GPU
Value printed from GPU: 5


## Steps for CUDA Programming:

- Load data to CPU memory
- Copy data from CPU to GPU memory
- Execute GPU kernel
- Copy results from CPU to GPU memory
- Use results on CPU

2. Write a CUDA Program to initialize an array of size N to all zeros in parallel.

## Recommended Launch Configuration:

**Choose a Number of Threads per Block:** A common choice is to use 256 or 512 threads per block. This number is a good balance between maximizing GPU occupancy and avoiding performance penalties associated with too many threads in a single block.

**Calculate the Number of Blocks:** To calculate the number of blocks needed, use the formula:

`numBlocks = 𝑁 + (threadsPerBlock − 1) / threadsPerBlock`


This formula ensures that you cover all elements in the array.

In [ ]:
%%writefile arrayInit.cu

#include <stdio.h>
#include <cuda.h>

__global__ void initArray(int N, int *arr) {
  int id = blockIdx.x * blockDim.x + threadIdx.x;
  if (id < N) { // Boundary check
    arr[id] = 0;
  }
}

int main() {
  int N = 10;
  int *arr;
  cudaMallocManaged(&arr, N * sizeof(int));

  // Launching the kernel with 1 block of N threads -- Sample launch (We'll see more about launch in next question)
  initArray<<<1, N>>>(N, arr);
  cudaDeviceSynchronize();

  // Printing the initialized array to confirm results
  for (int i = 0; i < N; i++) {
    printf("arr[%d] = %d\n", i, arr[i]);
  }

  // Free the allocated memory
  cudaFree(arr);
  return 0;
}

Writing arrayInit.cu


In [ ]:
!nvcc arrayInit.cu -o arrayInit
!./arrayInit

arr[0] = 0
arr[1] = 0
arr[2] = 0
arr[3] = 0
arr[4] = 0
arr[5] = 0
arr[6] = 0
arr[7] = 0
arr[8] = 0
arr[9] = 0


3. Add a scalar to all elements of array. (Compare CPU and GPU time)

In [ ]:
%%writefile scalarAdd.cu

#include <stdio.h>
#include <cuda.h>
#include <time.h>
#include <stdlib.h>

__global__ void scalarAdd_GPU(int *arr, int scalar, int N) {
    int id = blockIdx.x * blockDim.x + threadIdx.x; // Correct variable name for id
    if (id < N) {
      arr[id] += scalar;
    }
}

int *scalarAdd_CPU(int *arr, int scalar, int N) {
    for (int i = 0; i < N; i++) {
        arr[i] += scalar;
    }
    return arr;
}

int main() {
    int N = 100000; // Change this to a larger number for better performance testing
    int *arr;
    arr = (int *)malloc(N * sizeof(int)); // Added missing semicolon

    srand(time(NULL));
    for (int i = 0; i < N; i++) { // Use N instead of 1000 for the loop
        arr[i] = rand() % 100;
    }

    int scalar = 5;
    int *arr_GPU;
    cudaMallocManaged(&arr_GPU, N * sizeof(int));
    cudaMemcpy(arr_GPU, arr, N * sizeof(int), cudaMemcpyHostToDevice);

    // CPU timing
    printf("CPU time: \n");
    clock_t start = clock();
    int *arr_CPU = scalarAdd_CPU(arr, scalar, N);
    clock_t end = clock();
    printf("Time elapsed: %f seconds\n\n", (double)(end - start) / CLOCKS_PER_SEC);


    // Launch configuration
    int threadsPerBlock = 256; // Number of threads per block
    int numBlocks = (N + threadsPerBlock - 1) / threadsPerBlock; // Calculate number of blocks

    // GPU timing
    printf("GPU time: \n");
    cudaEvent_t startEvent, endEvent; // Renamed to avoid conflict with variables
    cudaEventCreate(&startEvent);
    cudaEventCreate(&endEvent);
    cudaEventRecord(startEvent);

    scalarAdd_GPU<<<numBlocks, threadsPerBlock>>>(arr_GPU, scalar, N); // Launching with multiple threads

    cudaEventRecord(endEvent);
    cudaEventSynchronize(endEvent); // Wait for the end event to complete

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, startEvent, endEvent); // Get elapsed time
    printf("Time elapsed: %f seconds\n\n", milliseconds / 1000.0); // Convert to seconds

    // Freeing memory
    cudaFree(arr_GPU);
    free(arr);
    cudaEventDestroy(startEvent);
    cudaEventDestroy(endEvent);

    return 0;
}

Writing scalarAdd.cu


In [ ]:
!nvcc scalarAdd.cu -o scalarAdd
!./scalarAdd

CPU time: 
Time elapsed: 0.000315 seconds

GPU time: 
Time elapsed: 0.057281 seconds



**Matrix -> Vector representation:**

M_mat = [[c0, c1, c2, c3], <br>
[c4, c5, c6, c7], <br>
[c8, c9, c10, c11], <br>
[c12, c13, c14, c15]]

is reresented as
M_vec = [c0, c1, ......., c15]

`M_vec[row * num_cols + col] = M_mat[row][col]`

4. Transpose a Matrix of size N

In [ ]:
%%writefile matrixTranpose.cu

#include <stdio.h>
#include <cuda.h>

__global__ void transpose(int *M_in, int *M_out, int rows, int cols) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;

    // Calculate row and column based on the 1D index
    int rowIdx = id / cols;
    int colIdx = id % cols;

    if (rowIdx < rows && colIdx < cols) {
        // Perform transpose by swapping rows and columns
        M_out[colIdx * rows + rowIdx] = M_in[rowIdx * cols + colIdx];
    }
}

int main() {
    int rows = 1000;
    int cols = 1000;
    int N = rows * cols;
    int M_in[N], M_out[N];

    // Initialize the input matrix (flattened as a vector)
    for (int i = 0; i < N; i++) {
        M_in[i] = i + 1;
    }

    int *d_M_in, *d_M_out;
    cudaMalloc(&d_M_in, N * sizeof(int));
    cudaMalloc(&d_M_out, N * sizeof(int));

    cudaMemcpy(d_M_in, M_in, N * sizeof(int), cudaMemcpyHostToDevice);

    // Launch configuration
    int threadsPerBlock = 256;
    int numBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Call the kernel
    transpose<<<numBlocks, threadsPerBlock>>>(d_M_in, d_M_out, rows, cols);

    // Copy result back to host
    cudaMemcpy(M_out, d_M_out, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print result
    printf("Transposed Matrix:\n");
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%d ", M_out[i * cols + j]);
        }
        printf("\n");
    }

    cudaFree(d_M_in);
    cudaFree(d_M_out);

    return 0;
}

Writing matrixTranpose.cu


In [ ]:
!nvcc matrixTranpose.cu -o matrixTranpose
!./matrixTranpose

Transposed Matrix:
1 5 9 13 
2 6 10 14 
3 7 11 15 
4 8 12 16 


5. Matrix Multiplication

In [ ]:
%%writefile matMult.cu

#include <stdio.h>
#include <cuda.h>

__global__ void matrixMultiply(int *A, int *B, int *C, int M, int K, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        int sum = 0;
        for (int k = 0; k < K; k++) {
            sum += A[row * K + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main() {
    int M = 4, K = 4, N = 4;
    int sizeA = M * K;
    int sizeB = K * N;
    int sizeC = M * N;

    int A[sizeA], B[sizeB], C[sizeC];

    // Initialize matrices A and B with random values
    for (int i = 0; i < sizeA; i++) A[i] = rand() % 10;
    for (int i = 0; i < sizeB; i++) B[i] = rand() % 10;

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, sizeA * sizeof(int));
    cudaMalloc(&d_B, sizeB * sizeof(int));
    cudaMalloc(&d_C, sizeC * sizeof(int));

    cudaMemcpy(d_A, A, sizeA * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, sizeB * sizeof(int), cudaMemcpyHostToDevice);

    // Configure grid and block dimensions
    dim3 threadsPerBlock(16, 16); // 16x16 threads per block
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Launch kernel
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, K, N);

    cudaMemcpy(C, d_C, sizeC * sizeof(int), cudaMemcpyDeviceToHost);

    // Display result
    printf("Matrix C (Result):\n");
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", C[i * N + j]);
        }
        printf("\n");
    }

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}

Writing matMult.cu


In [ ]:
!nvcc matMult.cu -o matMult
!./matMult

Matrix C (Result):
55 91 107 103 
31 68 71 85 
54 97 92 83 
57 102 123 102 


In [ ]:
%%writefile matrixVectorScale.cu

#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

// CUDA kernel for matrix to vector conversion and element-wise scaling
__global__ void matrixToVectorAndScale(const float* matrix, float* vector, float scale, int rows, int cols) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int size = rows * cols;

    if (idx < size) {
        vector[idx] = matrix[idx] * scale;
    }
}

int main() {
    // Example sizes
    int rows = 1 << 10; // 1024
    int cols = 1 << 10; // 1024
    int size = rows * cols;

    // Allocate host memory
    float *h_matrix = (float*)malloc(size * sizeof(float));
    float *h_vector = (float*)malloc(size * sizeof(float));

    // Initialize matrix
    for(int i = 0; i < size; i++) h_matrix[i] = static_cast<float>(i);

    // Allocate device memory
    float *d_matrix, *d_vector;
    cudaMalloc((void**)&d_matrix, size * sizeof(float));
    cudaMalloc((void**)&d_vector, size * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_matrix, h_matrix, size * sizeof(float), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

    // Launch kernel with scaling factor, e.g., scale by 2.0
    float scale = 2.0f;
    matrixToVectorAndScale<<<blocksPerGrid, threadsPerBlock>>>(d_matrix, d_vector, scale, rows, cols);

    // Copy result back to host
    cudaMemcpy(h_vector, d_vector, size * sizeof(float), cudaMemcpyDeviceToHost);

    // Cleanup
    cudaFree(d_matrix);
    cudaFree(d_vector);
    free(h_matrix);
    free(h_vector);

    return 0;
}


Writing matrixVectorScale.cu


In [ ]:
!nvcc matrixVectorScale.cu -o matrixVectorScale
!./matrixVectorScale

In [ ]:
!nvprof ./matrixVectorScale

==792== NVPROF is profiling process 792, command: ./matrixVectorScale
==792== Profiling application: ./matrixVectorScale
==792== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   70.24%  1.9928ms         1  1.9928ms  1.9928ms  1.9928ms  [CUDA memcpy DtoH]
                   28.32%  803.60us         1  803.60us  803.60us  803.60us  [CUDA memcpy HtoD]
                    1.43%  40.703us         1  40.703us  40.703us  40.703us  matrixToVectorAndScale(float const *, float*, float, int, int)
      API calls:   94.11%  88.741ms         2  44.370ms  120.85us  88.620ms  cudaMalloc
                    5.01%  4.7233ms         2  2.3616ms  977.37us  3.7459ms  cudaMemcpy
                    0.38%  357.18us         2  178.59us  134.56us  222.62us  cudaFree
                    0.26%  245.47us         1  245.47us  245.47us  245.47us  cudaLaunchKernel
                    0.19%  182.47us       114  1.6000us     190ns  71.371us  cuDevi

In [ ]:
%%writefile nonlinear_elementwise_with_printf.cu

#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>
#include <math.h>

// Example: Sigmoid function approximation using a Taylor series (for illustration)
__device__ float sigmoid(float x) {
    // Simple approximation: sigmoid(x) ≈ x / (1 + |x|)
    return x / (1.0f + fabsf(x));
}

__global__ void applyNonLinearFunction(const float* input, float* output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < size) {
        output[idx] = sigmoid(input[idx]);

        // Print intermediate non-linear application results for debugging
        printf("Thread %d: input[%d] = %.2f, sigmoid ≈ %.2f\n", idx, idx, input[idx], output[idx]);
    }
}

int main() {
    // Example vector size
    int size = 8;

    // Allocate host memory
    float h_input[] = {-3.0f, -1.0f, 0.0f, 1.0f, 2.0f, 3.0f, 4.0f, 5.0f};
    float h_output[size] = {0};

    // Allocate device memory
    float *d_input, *d_output;
    cudaMalloc((void**)&d_input, size * sizeof(float));
    cudaMalloc((void**)&d_output, size * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_input, h_input, size * sizeof(float), cudaMemcpyHostToDevice);

    // Define block and grid sizes
    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

    // Launch kernel
    applyNonLinearFunction<<<blocksPerGrid, threadsPerBlock>>>(d_input, d_output, size);

    // Wait for GPU to finish
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(h_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);

    // Print the output vector after applying non-linear function
    printf("\nOutput Vector after Sigmoid Approximation:\n");
    for(int i = 0; i < size; i++) {
        printf("output[%d] = %.2f\n", i, h_output[i]);
    }

    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

Writing nonlinear_elementwise_with_printf.cu


In [ ]:
!nvcc nonlinear_elementwise_with_printf.cu -o nonlinear_elementwise_with_printf
!./nonlinear_elementwise_with_printf

Thread 0: input[0] = -3.00, sigmoid ≈ -0.75
Thread 1: input[1] = -1.00, sigmoid ≈ -0.50
Thread 2: input[2] = 0.00, sigmoid ≈ 0.00
Thread 3: input[3] = 1.00, sigmoid ≈ 0.50
Thread 4: input[4] = 2.00, sigmoid ≈ 0.67
Thread 5: input[5] = 3.00, sigmoid ≈ 0.75
Thread 6: input[6] = 4.00, sigmoid ≈ 0.80
Thread 7: input[7] = 5.00, sigmoid ≈ 0.83

Output Vector after Sigmoid Approximation:
output[0] = -0.75
output[1] = -0.50
output[2] = 0.00
output[3] = 0.50
output[4] = 0.67
output[5] = 0.75
output[6] = 0.80
output[7] = 0.83


In [ ]:
!nvprof ./nonlinear_elementwise_with_printf

==2305== NVPROF is profiling process 2305, command: ./nonlinear_elementwise_with_printf
Thread 0: input[0] = -3.00, sigmoid ≈ -0.75
Thread 1: input[1] = -1.00, sigmoid ≈ -0.50
Thread 2: input[2] = 0.00, sigmoid ≈ 0.00
Thread 3: input[3] = 1.00, sigmoid ≈ 0.50
Thread 4: input[4] = 2.00, sigmoid ≈ 0.67
Thread 5: input[5] = 3.00, sigmoid ≈ 0.75
Thread 6: input[6] = 4.00, sigmoid ≈ 0.80
Thread 7: input[7] = 5.00, sigmoid ≈ 0.83

Output Vector after Sigmoid Approximation:
output[0] = -0.75
output[1] = -0.50
output[2] = 0.00
output[3] = 0.50
output[4] = 0.67
output[5] = 0.75
output[6] = 0.80
output[7] = 0.83
==2305== Profiling application: ./nonlinear_elementwise_with_printf
==2305== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.05%  110.46us         1  110.46us  110.46us  110.46us  applyNonLinearFunction(float const *, float*, int)
                    1.91%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA me

Use Thrust library to perform matrix-vector multiplication

In [ ]:
#include <iostream>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/sequence.h>

struct matrix_vector_mult {
    const float* matrix;
    const float* vector;
    int cols;

    matrix_vector_mult(const float* m, const float* v, int c) : matrix(m), vector(v), cols(c) {}

    __device__ float operator()(int row) {
        float sum = 0;
        for (int i = 0; i < cols; i++) {
            sum += matrix[row * cols + i] * vector[i];
        }
        return sum;
    }
};

int main() {
    const int rows = 3, cols = 4;
    float h_matrix[rows * cols] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12};
    float h_vector[cols] = {1, 1, 1, 1};
    thrust::device_vector<float> d_matrix(h_matrix, h_matrix + rows * cols);
    thrust::device_vector<float> d_vector(h_vector, h_vector + cols);
    thrust::device_vector<float> d_output(rows);

    thrust::transform(thrust::counting_iterator<int>(0), thrust::counting_iterator<int>(rows), d_output.begin(),
                      matrix_vector_mult(thrust::raw_pointer_cast(d_matrix.data()), thrust::raw_pointer_cast(d_vector.data()), cols));

    std::cout << "Thrust Matrix-Vector Multiplication Output:\n";
    for (int i = 0; i < rows; i++) {
        std::cout << d_output[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}
